In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
train_data = 'C:\\Users\\Lakshmi Praffulla\\Desktop\\MLBA_Kaggle\\train.csv'
test_data = "C:\\Users\\Lakshmi Praffulla\\Desktop\\MLBA_Kaggle\\valid.csv"

In [3]:
def getData(path):
    label = []
    
    with open(train_data) as f:
        sequence = []
        label_str = []
        for line in f:
            text_file = line.split(",");
            if text_file[0] == 'ID':
                continue;
            label_str.append(text_file[1])
            s = text_file[2]
            sequence.append(s[:-1]);
    
    for i in label_str:
        if i == 'DNA':
            label.append(1)
        else:
            label.append(0)
    
    return sequence,label

In [4]:
x_train,y_train = getData(train_data);

In [5]:
len(x_train)

3049

In [6]:
aminoacid_sequence = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

tripeptide_seq = []

for charone in aminoacid_sequence:
    for chartwo in aminoacid_sequence:
        for charthree in aminoacid_sequence:
            char_to_add = charone+chartwo+charthree
            tripeptide_seq.append(char_to_add)

In [9]:
len(tripeptide_seq)

8000

In [20]:
def get_dipeptite_dictionary(str):
    
    size = len(str)
    i = 0
    
    eachseq_list = []
    while( i < size - 2 ):
        add_seq = str[i] + str[i+1] + str[i+2]
        eachseq_list.append(add_seq)
        i += 1
    
    getdict = {}
    
    for item in eachseq_list:
        if getdict.get(item) == None:
            getdict[item] = 1
        else:
            temp = getdict[item]
            temp += 1
            getdict[item] = temp
    
    new_dict = {}
    for item in getdict:
        value = getdict[item]
        value = value/(len(str)-2)
        new_dict[item] = value
    
    return new_dict

In [21]:
def getlist_of_dictionary(x_train):
    list_dictionary = []
    
    for each in x_train:
        dic = get_dipeptite_dictionary(each)
        list_dictionary.append(dic)
        
    return list_dictionary

In [33]:
def getvector(vector_dictionary):
    final_vector = []
    
    count = 0
    for dic in vector_dictionary:
        print(count)
        count += 1
        vec = [0.0] * 8000
        for item in dic:
            for seq in tripeptide_seq:
                if seq == item:
                    index = tripeptide_seq.index(seq)
                    vec[index] = dic[item]
                    
        final_vector.append(vec)            
    
    return final_vector

In [28]:
training_vector_dict = getlist_of_dictionary(x_train)

In [1]:
X_Train = getvector(training_vector_dict)

In [32]:
# len(training_vector_dict)

3049

In [35]:
x = np.array(X_Train)
y = np.array(y_train)

In [36]:
x.shape

(3049, 8000)

In [37]:
y.shape

(3049,)

In [38]:
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.3, random_state=42)

Test Data

In [39]:
ID = []
test_sequence = []

with open(test_data) as f:
    for line in f:
        after_split = line.split(",")
        if after_split[0] == 'ID':
            continue;
        ID.append(after_split[0])
        test_sequence.append(after_split[1][:-1])

In [40]:
len(ID)

1071

In [41]:
testing_vector_dict = getlist_of_dictionary(test_sequence)

In [42]:
len(testing_vector_dict)

1071

In [1]:
x_test = getvector(testing_vector_dict)

In [45]:
x_test = np.array(x_test)

In [47]:
x_test.shape

(1071, 8000)

Random Forest

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
clf_rfi = RandomForestClassifier()

In [50]:
clf_rfi.fit(x_train,y_train)

C:\Users\Lakshmi Praffulla\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
accuracy_score(y_train,clf_rfi.predict(x_train))

0.9868791002811621

In [52]:
accuracy_score(y_val,clf_rfi.predict(x_val))

0.6087431693989072

In [ ]:
ans_rfi = clf_rfi.predict(x_test)

Feature Reduction

In [63]:
from sklearn.decomposition import PCA

pca = PCA(n_components=500)

x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)


In [72]:
x_val_pca = pca.transform(x_val)

KNN

In [64]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=999)

In [85]:
import numpy as np
params_KNN = {'n_neighbors': [1,2,3,4,5,6], 
              'p': [1]}

In [86]:
from sklearn.model_selection import GridSearchCV

gs_KNN = GridSearchCV(estimator=KNeighborsClassifier(), 
                      param_grid=params_KNN, 
                      cv=cv_method,
                      verbose=100,  # verbose: the higher, the more messages
                      scoring='accuracy', 
                      return_train_score=True)

In [87]:
gs_KNN.fit(x_train, y_train);

Fitting 15 folds for each of 1 candidates, totalling 15 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] n_neighbors=4, p=1 ..............................................
[CV]  n_neighbors=4, p=1, score=(train=0.779, test=0.663), total=  15.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.2min remaining:    0.0s
[CV] n_neighbors=4, p=1 ..............................................
[CV]  n_neighbors=4, p=1, score=(train=0.756, test=0.642), total=  15.2s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.4min remaining:    0.0s
[CV] n_neighbors=4, p=1 ..............................................
[CV]  n_neighbors=4, p=1, score=(train=0.746, test=0.623), total=  14.7s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.6min remaining:    0.0s
[CV] n_neighbors=4, p=1 ..............................................
[CV]  n_neighbors=4, p=1, score=(train=0.767, test=0.667), total=  15.4s
[Parallel(n_jobs=1)]: Done   4 out of   4 

In [88]:
gs_KNN.best_params_

{'n_neighbors': 4, 'p': 1}

In [89]:
gs_KNN.best_score_

0.6537019681349578

In [91]:
accuracy_score(y_val,gs_KNN.predict(x_val))

0.673224043715847

In [92]:
ans_KNN = gs_KNN.predict(x_test)

In [93]:
# Writing onto a file to upload in kaggle.

f = open('KNN_Trip_67val.csv','w')

s = "ID,Lable\n"

c = 0
for i in ans_KNN:
    
    if i == 1:
        s = s+ID[c]+","+str(i)+"\n"
    else:
        s = s+ID[c]+","+str(-1)+"\n"
    c = c + 1

f.write(s)
f.close()